# 04 Basic Relevance Closure

This notebook defines a method to calculate if a statement (a twiddle statement) is entailed by the basic relevance closure of a knowledge base.

In [1]:
import os
import sys
from typing import Tuple, FrozenSet, Set

from more_itertools import powerset

sys.path.append(os.path.join('..', 'common'))

from datatypes import KnowledgeBase, Literal, Atom, Top, Bot, Normally, Formula
from util import materialized, print_knowledge_base, entails
from ranked_models import statement_ranking

In [2]:
bot = Bot()            # Falsum
top = Top()            # Verum
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus
s = Literal(Atom('s')) # mammalian sickle cells
b = Literal(Atom('b')) # bioconcave shape

This algorithm uses the subroutine `justifications` to find the subset of defeasible statements that cause a statement to be false.

In [3]:
def separate(knowledge_base: KnowledgeBase) -> Tuple[KnowledgeBase, KnowledgeBase]:
    T = {statement for statement in knowledge_base if statement.is_classical}
    return T, knowledge_base - T


def justifications(classical_statements: KnowledgeBase, defeasible_statements: KnowledgeBase, formula: Formula) -> Set[
    FrozenSet[Normally]]:
    max_size = float('inf')
    js = set()
    for c in powerset(defeasible_statements):
        if len(c) <= max_size:
            candidate: FrozenSet[Formula] = set(c)
            if entails(materialized(candidate | classical_statements), -formula):
                if len(candidate) < max_size:
                    js.clear()
                max_size = len(candidate)
                js.add(frozenset(candidate))

    return js

Basic relevance closure uses the union of the justifications and removes all statements (level by level) that are included in the union.

In [4]:
def basic_relevance_closure(knowledge_base: KnowledgeBase, statement: Normally) -> bool:
    rank = statement_ranking(knowledge_base)
    i = 0
    T, D = separate(knowledge_base)
    Rel = {j for justification in justifications(T, D, statement.left) for j in justification}
    Irr = D - Rel
    R_ = materialized(Rel)
    while R_ and entails(materialized(T) | materialized(Irr) | R_, -statement.left):
        R_ = R_ - materialized(rank[i] & Rel)
        i += 1
    return entails(materialized(T) | materialized(Irr) | R_, statement.materialize())

In [5]:
K = {-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n}
T, D = separate(K)
print_knowledge_base(K)

{ v |~ n, m |~ ¬n, v |~ c, m → v, a → v }


We can see that $a$ (avian blood cells) do not have a subset of statements that would render it false.

In [6]:
justifications(T, D, Formula(a))

set()

In [7]:
basic_relevance_closure(K, a/c)

True

However $m$ (mammalian blood cells) can be false, because of $m |\hspace{-0.5em}\sim{} \neg n$ and $m |\hspace{-0.5em}\sim{} n$.

In [8]:
justifications(T, D, Formula(m))

{frozenset({m |~ ¬n, v |~ n})}

In [9]:
basic_relevance_closure(K, m/c)

True

In [10]:
K={-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n, -(s >> m) / bot, m / b, s / -b}
T, D = separate(K)
print_knowledge_base(K)

{ v |~ n, m |~ ¬n, m |~ b, s |~ ¬b, v |~ c, m → v, s → m, a → v }


The justifications of a statement can also be multiple subsets.

In [11]:
justifications(T, D, Formula(s))

{frozenset({m |~ b, s |~ ¬b}), frozenset({m |~ ¬n, v |~ n})}

In [12]:
basic_relevance_closure(K, s / -n)

False